In [1]:
"""Imports"""
from calendar import month
from ctypes import c_ulong
import pandas as pd
import numpy as np
import re
import chainladder as cl
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import ipywidgets as widgets
from dateutil import relativedelta as rd
import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook

In [2]:
# Cargar el archivo Excel en un DataFrame
aamg_medical = pd.read_excel("2.AAMG IBNR - Medi-Cal.xlsx", sheet_name='Raw Triangle Accounting Model')
aamg_commercial = pd.read_excel("1.AAMG IBNR - Commercial.xlsx", sheet_name='Raw Triangle Accounting Model')
aamg_medicare = pd.read_excel("3.AAMG IBNR - Medicare.xlsx", sheet_name='Raw Triangle Accounting Model')
apc_commercial = pd.read_excel("4.APC IBNR Commercial.xlsx", sheet_name='Raw Triangle Accounting Model')
apc_medical = pd.read_excel("5.APC IBNR Medi-Cal.xlsx", sheet_name='Raw Triangle Accounting Model')
apc_medicare = pd.read_excel("6.APC IBNR Medicare.xlsx", sheet_name='Raw Triangle Accounting Model')
cfc_commercial = pd.read_excel("7.CFC IBNR - Commercial.xlsx", sheet_name='Raw Triangle Accounting Model')
cfc_medical = pd.read_excel("8.CFC IBNR - Medi-Cal.xlsx", sheet_name='Raw Triangle Accounting Model')
cfc_medicare = pd.read_excel("9.CFC IBNR - Medicare.xlsx", sheet_name='Raw Triangle Accounting Model')


# Seleccionar un rango específico de filas y columnas
aamg_medical = aamg_medical.iloc[4:41, 1:38]
aamg_commercial = aamg_commercial.iloc[4:41, 1:38]
aamg_medicare = aamg_medicare.iloc[4:41, 1:38]
apc_commercial = apc_commercial.iloc[4:41, 1:38]
apc_medical = apc_medical.iloc[4:41, 1:38]
apc_medicare = apc_medicare.iloc[4:41, 1:38]
cfc_commercial = cfc_commercial.iloc[4:41, 1:38]
cfc_medical = cfc_medical.iloc[4:41, 1:38]
cfc_medicare = cfc_medicare.iloc[4:41, 1:38]

# Obtener la primera fila del DataFrame como los nuevos nombres de columnas --> la primera fila en este caso son los AY del triángulo
new_columns1 = aamg_medical.iloc[0]
new_columns2 = aamg_commercial.iloc[0]
new_columns3 = aamg_medicare.iloc[0]
new_columns4 = apc_commercial.iloc[0]
new_columns5 = apc_medical .iloc[0]
new_columns6 = apc_medicare.iloc[0]
new_columns7 = cfc_commercial.iloc[0]
new_columns8 = cfc_medical.iloc[0]
new_columns9 = cfc_medicare.iloc[0]

# Asignar los nuevos nombres de columnas al DataFrame --> nombramos como columnas a los AY
aamg_medical.columns = new_columns1
aamg_commercial.columns = new_columns2
aamg_medicare.columns = new_columns3
apc_commercial.columns = new_columns4
apc_medical .columns = new_columns5
apc_medicare.columns = new_columns6
cfc_commercial.columns = new_columns7
cfc_medical.columns = new_columns8
cfc_medicare.columns = new_columns9

# Eliminar la primera fila (que ahora son los nombres de las columnas) del DataFrame
aamg_medical = aamg_medical[1:]
aamg_commercial = aamg_commercial[1:]
aamg_medicare = aamg_medicare[1:]
apc_commercial = apc_commercial[1:]
apc_medical = apc_medical[1:]
apc_medicare = apc_medicare[1:]
cfc_commercial = cfc_commercial[1:]
cfc_medical = cfc_medical[1:]
cfc_medicare = cfc_medicare[1:]


# Función para descomponer el triángulo en formato largo
def descomponer_triangulo(triangulo_df, segmento):
    triangulo_long = triangulo_df.melt(id_vars=triangulo_df.columns[0], #"melt" en pandas convierte de formato ancho a largo, en este caso la primera columna
                                       var_name='Acc_Month', #--> el resto las convierte como ay  y amount
                                       value_name='ClaimAmount')
    triangulo_long['Segment'] = segmento ##--> Nombre de la columna que agregamos
    triangulo_long.rename(columns={triangulo_df.columns[0]: 'Val_Month'}, inplace=True)
   
    return triangulo_long

# Descomponer  triángulo --> toma un dataframe y lo transforma de un formato ancho a un formato largo, agregando la columna de segmento y renombrando AY y Devyear
aamg_medical_long = descomponer_triangulo(aamg_medical, 'AAMG Medi-Cal')
aamg_commercial_long = descomponer_triangulo(aamg_commercial, 'AAMG Commercial')
aamg_medicare_long = descomponer_triangulo(aamg_medicare, 'AAMG Medicare')
apc_commercial_long = descomponer_triangulo(apc_commercial, 'APC Commercial')
apc_medical_long = descomponer_triangulo(apc_medical, 'APC Medi-Cal')
apc_medicare_long = descomponer_triangulo(apc_medicare, 'APC Medicare')
cfc_commercial_long = descomponer_triangulo(cfc_commercial, 'CFC Commercial')
cfc_medical_long = descomponer_triangulo(cfc_medical, 'CFC Medi-Cal')
cfc_medicare_long = descomponer_triangulo(cfc_medicare, 'CFC Medicare')


# Función para agregar columnas adicionales
def agregar_columnas_adicionales(triangulo_long):
    triangulo_long['AccMonthSt'] = triangulo_long['Acc_Month'] #--> agregamos una nueva columna con el mismo valor que accmonth, simplemente duplica la columna
    triangulo_long['AccMonthEnd'] = triangulo_long['Acc_Month']
   
    # Convertir Acc_Month a fechas
    triangulo_long['Acc_Month'] = pd.to_datetime(triangulo_long['Acc_Month'], format='%Y%m')
         
    # Convertir Val_Month a fechas
    triangulo_long['Val_Month'] = pd.to_datetime(triangulo_long['Val_Month'], format='%Y%m')
   
    return triangulo_long

# Agregar columnas adicionales a ambos DataFrames
aamg_medical_long = agregar_columnas_adicionales(aamg_medical_long)
aamg_commercial_long = agregar_columnas_adicionales(aamg_commercial_long)
aamg_medicare_long = agregar_columnas_adicionales(aamg_medicare_long)
apc_commercial_long = agregar_columnas_adicionales(apc_commercial_long)
apc_medical_long = agregar_columnas_adicionales(apc_medical_long)
apc_medicare_long = agregar_columnas_adicionales(apc_medicare_long)
cfc_commercial_long = agregar_columnas_adicionales(cfc_commercial_long)
cfc_medical_long = agregar_columnas_adicionales(cfc_medical_long)
cfc_medicare_long = agregar_columnas_adicionales(cfc_medicare_long)



FileNotFoundError: [Errno 2] No such file or directory: '2.AAMG IBNR - Medi-Cal.xlsx'

In [ ]:
aamg_commercial_long

,Val_Month,Acc_Month,ClaimAmount,Segment,AccMonthSt,AccMonthEnd
0,2021-04-01,2021-04-01,48995.19,AAMG Commercial,202104,202104
1,2021-05-01,2021-04-01,56633.46,AAMG Commercial,202104,202104
2,2021-06-01,2021-04-01,35741.19,AAMG Commercial,202104,202104
3,2021-07-01,2021-04-01,3955.09,AAMG Commercial,202104,202104
4,2021-08-01,2021-04-01,3954.61,AAMG Commercial,202104,202104
...,...,...,...,...,...,...
1291,2023-11-01,2024-03-01,0,AAMG Commercial,202403,202403
1292,2023-12-01,2024-03-01,0,AAMG Commercial,202403,202403
1293,2024-01-01,2024-03-01,0,AAMG Commercial,202403,202403
1294,2024-02-01,2024-03-01,0,AAMG Commercial,202403,202403


In [ ]:

#Armamos los triángulos

aamg_medical_triangle = cl.Triangle(
    aamg_medical_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

aamg_commercial_triangle = cl.Triangle(
    aamg_commercial_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

aamg_medicare_triangle = cl.Triangle(
    aamg_medicare_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

apc_commercial_triangle = cl.Triangle(
    apc_commercial_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

apc_medical_triangle = cl.Triangle(
    apc_medical_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

apc_medicare_triangle = cl.Triangle(
    apc_medicare_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

cfc_commercial_triangle = cl.Triangle(
    cfc_commercial_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

cfc_medical_triangle = cl.Triangle(
    cfc_medical_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

cfc_medicare_triangle = cl.Triangle(
    cfc_medicare_long,
    origin='Acc_Month',
    development='Val_Month',
    columns='ClaimAmount',
    index=["Segment"],
    cumulative=True  # Explicitly set cumulative property to True
)

# Obtener el DataFrame del triángulo (facilita mucho mas el manejo de los datos)
aamg_medical_triangle = aamg_medical_triangle.to_frame()
aamg_commercial_triangle = aamg_commercial_triangle.to_frame()
aamg_medicare_triangle = aamg_medicare_triangle.to_frame()
apc_medical_triangle = apc_medical_triangle.to_frame()
apc_commercial_triangle = apc_commercial_triangle.to_frame()
apc_medicare_triangle = apc_medicare_triangle.to_frame()
cfc_commercial_triangle = cfc_commercial_triangle.to_frame()
cfc_medical_triangle = cfc_medical_triangle.to_frame()
cfc_medicare_triangle = cfc_medicare_triangle.to_frame()

# Obtener los meses únicos de Acc_Month
meses1 = aamg_medical_long['Acc_Month'].unique()
meses2 = aamg_commercial_long['Acc_Month'].unique()
meses3 = aamg_medicare_long['Acc_Month'].unique()
meses4 = apc_medical_long['Acc_Month'].unique()
meses5 = apc_commercial_long['Acc_Month'].unique()
meses6 = apc_medicare_long['Acc_Month'].unique()
meses7 = cfc_commercial_long['Acc_Month'].unique()
meses8 = cfc_medical_long['Acc_Month'].unique()
meses9 = cfc_medicare_long['Acc_Month'].unique()

# Formatear los meses para las filas
nuevos_nombres_filas1 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses1]
nuevos_nombres_filas2 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses2]
nuevos_nombres_filas3 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses3]
nuevos_nombres_filas4 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses4]
nuevos_nombres_filas5 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses5]
nuevos_nombres_filas6 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses6]
nuevos_nombres_filas7 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses7]
nuevos_nombres_filas8 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses8]
nuevos_nombres_filas9 = [pd.to_datetime(mes).strftime('%b-%y') for mes in meses9]

# Cambiar nombres de filas --> solo por una cuestion de que quede con el mismo formato qcon el que se pega en el excel desp
aamg_medical_triangle.index = nuevos_nombres_filas1
aamg_commercial_triangle.index = nuevos_nombres_filas2
aamg_medicare_triangle.index = nuevos_nombres_filas3
apc_medical_triangle.index = nuevos_nombres_filas4
apc_commercial_triangle.index = nuevos_nombres_filas5
apc_medicare_triangle.index = nuevos_nombres_filas6
cfc_commercial_triangle.index = nuevos_nombres_filas7
cfc_medical_triangle.index = nuevos_nombres_filas8
cfc_medicare_triangle.index = nuevos_nombres_filas9


# Cambiar nombres de columnas
num_columnas1 = aamg_medical_triangle.shape[1] #--> segun el nro de columnas total
nuevos_nombres_columnas1 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas1)]
aamg_medical_triangle.columns = nuevos_nombres_columnas1

num_columnas2 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas2 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas2)]
aamg_commercial_triangle.columns = nuevos_nombres_columnas2

num_columnas3 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas3 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas3)]
aamg_medicare_triangle.columns = nuevos_nombres_columnas3

num_columnas4 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas4 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas4)]
apc_medical_triangle.columns = nuevos_nombres_columnas4

num_columnas5 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas5 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas5)]
apc_commercial_triangle.columns = nuevos_nombres_columnas5

num_columnas6 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas6 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas6)]
apc_medicare_triangle.columns = nuevos_nombres_columnas6

num_columnas7 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas7 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas7)]
cfc_commercial_triangle.columns = nuevos_nombres_columnas7

num_columnas8 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas8 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas8)]
cfc_medical_triangle.columns = nuevos_nombres_columnas8

num_columnas9 = aamg_commercial_triangle.shape[1]
nuevos_nombres_columnas9 = [f'Inc Mo +{i}' if i > 0 else 'Inc Mo' for i in range(num_columnas9)]
cfc_medicare_triangle.columns = nuevos_nombres_columnas9

# Transponer el DataFrame
aamg_medical_triangle = aamg_medical_triangle.T
aamg_commercial_triangle = aamg_commercial_triangle.T
aamg_medicare_triangle = aamg_medicare_triangle.T
apc_medical_triangle = apc_medical_triangle.T
apc_commercial_triangle = apc_commercial_triangle.T
apc_medicare_triangle = apc_medicare_triangle.T
cfc_commercial_triangle = cfc_commercial_triangle.T
cfc_medical_triangle = cfc_medical_triangle.T
cfc_medicare_triangle = cfc_medicare_triangle.T

#Exportamos a un excel con distintas hojas

with pd.ExcelWriter('IBNP Triangles.xlsx', engine='openpyxl') as writer:
    df1 = pd.DataFrame(aamg_medical_triangle)
    df1.to_excel(writer, sheet_name='AAMG Medi-Cal', index=True)
    
    df2 = pd.DataFrame(aamg_commercial_triangle)
    df2.to_excel(writer, sheet_name='AAMG Commercial', index=True)

    df3 = pd.DataFrame(aamg_medicare_triangle)
    df3.to_excel(writer, sheet_name='AAMG Medicare', index=True)

    df4 = pd.DataFrame(apc_medical_triangle)
    df4.to_excel(writer, sheet_name='APC Medi-Cal', index=True)

    df5 = pd.DataFrame(apc_commercial_triangle)
    df5.to_excel(writer, sheet_name='APC Commercial', index=True)

    df6 = pd.DataFrame(apc_medicare_triangle)
    df6.to_excel(writer, sheet_name='APC Medicare', index=True)

    df7 = pd.DataFrame(cfc_commercial_triangle)
    df7.to_excel(writer, sheet_name='CFC Commercial', index=True)

    df8 = pd.DataFrame(cfc_medical_triangle)
    df8.to_excel(writer, sheet_name='CFC Medi-Cal', index=True)

    df9 = pd.DataFrame(cfc_medicare_triangle)
    df9.to_excel(writer, sheet_name='CFC Medicare', index=True)



c:\Users\DPecina\AppData\Local\anaconda3\Lib\site-packages\chainladder\core\base.py:100: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_agg = data[columns].groupby(key_gr).sum().reset_index().fillna(0)
c:\Users\DPecina\AppData\Local\anaconda3\Lib\site-packages\chainladder\core\base.py:106: UserWarning: 
                Observations with development before
                origin start have been removed.
  warnings.warn(
c:\Users\DPecina\AppData\Local\anaconda3\Lib\site-packages\chainladder\core\base.py:100: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [ ]:
aamg_medical_triangle

,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,...,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23,Jan-24,Feb-24,Mar-24
Inc Mo,15475.39,32036.01,13250.84,16187.23,14349.38,21217.54,7550.34,12392.53,18493.71,13295.17,...,40550.73,43824.74,97647.47,43579.71,25706.91,39279.30,31889.31,44497.24,56183.40,39963.09
Inc Mo +1,13580.83,79150.72,47247.21,24613.07,49619.84,18643.40,50078.89,38105.97,28796.04,21977.93,...,147953.95,186806.59,78811.78,108101.08,182843.51,145952.35,142928.67,169170.10,73477.61,NaN
Inc Mo +2,2670.24,18623.05,1974.42,7279.60,1912.96,6423.35,3991.19,4001.92,1637.94,3744.98,...,35889.68,21064.64,66019.10,43995.79,43359.05,68360.77,39184.37,24106.75,NaN,NaN
Inc Mo +3,2117.08,2592.17,2381.19,1695.57,1569.32,2565.41,2305.21,2829.82,2470.72,480.98,...,11507.55,15661.64,19718.05,10959.05,16177.32,26492.63,19436.60,NaN,NaN,NaN
Inc Mo +4,480.25,2216.84,451.14,652.54,813.82,379.69,1527.34,1021.61,312.77,273.06,...,7390.98,17356.99,12334.35,7098.09,7196.04,1974.74,NaN,NaN,NaN,NaN
Inc Mo +5,832.56,398.09,123.60,749.11,110.64,246.30,127.15,492.92,235.15,1072.38,...,5123.94,6375.68,7298.21,8861.94,6162.98,NaN,NaN,NaN,NaN,NaN
Inc Mo +6,102.96,324.80,282.07,593.12,181.06,2627.90,148.46,226.42,3400.15,31.36,...,2572.38,8361.71,7199.63,3862.53,NaN,NaN,NaN,NaN,NaN,NaN
Inc Mo +7,NaN,969.18,874.45,NaN,43.20,625.41,123.46,2529.75,123.46,NaN,...,2072.41,1325.47,2423.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Inc Mo +8,416.66,368.43,408.90,348.56,NaN,NaN,5303.11,107.69,443.65,32.58,...,2912.18,2139.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Inc Mo +9,NaN,271.14,172.35,NaN,59.08,100.00,303.96,NaN,68.11,43.20,...,8899.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
